In [1]:
import os

In [2]:
os.chdir("../../")

In [3]:
%pwd

'/Users/bhikipallai/Desktop/Projects/95Mobiles'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class CarModelevalConfig:
    root_dir: Path
    X_train: Path
    X_test: Path
    y_train:Path
    y_test: Path
    model_for_train: Path
    save_score: Path

In [5]:
from src.car.constants import *
from src.car.utils.common import read_yaml,create_directory

[2024-10-01 13:24:31,300: INFO]: Logging file start


In [6]:
class ConfigurationManger:
    def __init__(self,config_filepath = CONFIG_FILE_PATH,params_filepath = PARAM_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directory([self.config.data_root])
    
    def get_model_eval(self) -> CarModelevalConfig:
         config = self.config.model_eval
         create_directory([config.root_dir])

         modelevalconfig = CarModelevalConfig(
             root_dir=config.root_dir,
             X_train=config.X_train,
             X_test = config.X_test,
             y_train=config.y_train,
             y_test = config.y_test,
             model_for_train = config.model_for_train,
             save_score = config.save_score
         )

         return modelevalconfig

In [7]:
from sklearn.metrics import r2_score
import pandas as pd
import joblib
import json
import dagshub
import mlflow

In [12]:
class ModelEval:
    def __init__(self,config:CarModelevalConfig):
        self.config = config

    def model_eval(self):
        X_train = pd.read_csv(self.config.X_train)
        X_test = pd.read_csv(self.config.X_test)
        y_train = pd.read_csv(self.config.y_train)
        y_test = pd.read_csv(self.config.y_test)


        model = joblib.load(self.config.model_for_train)

        dagshub.init(repo_owner='Vicky7873', repo_name='95Mobiles', mlflow=True)
        mlflow.set_registry_uri("https://dagshub.com/Vicky7873/95Mobiles.mlflow")
        mlflow.set_experiment("Car Model Eval")

        with mlflow.start_run():
            print("train acc: ",model.score(X_train,y_train))
            train_score = model.score(X_train,y_train)
            y_pred = model.predict(X_test)
            test_acc = r2_score(y_test,y_pred)
            print("test acc: ", test_acc )

            score = {
                "train score: ": train_score,
                "test score: ": test_acc
            }
            mlflow.log_metric("Training_acc",train_score)
            mlflow.log_metric("Testing_acc",test_acc)

        with open (self.config.save_score,"w") as file:
            json.dump(score,file,indent=4)

In [13]:
try:
    config = ConfigurationManger()
    eval_con = config.get_model_eval()
    model_eval = ModelEval(config=eval_con)
    model_eval.model_eval()
except Exception as e:
    raise e

[2024-10-01 13:26:59,644: INFO]: yaml: config/car_config.yaml loaded successfully
[2024-10-01 13:26:59,646: INFO]: yaml: params/car_params.yaml loaded successfully
[2024-10-01 13:26:59,647: INFO]: created directory at: data
[2024-10-01 13:26:59,647: INFO]: created directory at: data/car/model_eval
[2024-10-01 13:27:00,094: INFO]: HTTP Request: GET https://dagshub.com/api/v1/repos/Vicky7873/95Mobiles "HTTP/1.1 200 OK"


Initialized MLflow to track repo "Vicky7873/95Mobiles"

[2024-10-01 13:27:00,101: INFO]: Initialized MLflow to track repo "Vicky7873/95Mobiles"


Repository Vicky7873/95Mobiles initialized!

[2024-10-01 13:27:00,103: INFO]: Repository Vicky7873/95Mobiles initialized!
train acc:  0.9492088292736656
test acc:  0.873602758632099


2024/10/01 13:27:02 INFO mlflow.tracking._tracking_service.client: 🏃 View run enthused-croc-186 at: https://dagshub.com/Vicky7873/95Mobiles.mlflow/#/experiments/4/runs/04201504bea54469a86a3a75bce580f2.
2024/10/01 13:27:02 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/Vicky7873/95Mobiles.mlflow/#/experiments/4.
